In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
def evolve(x): 
    # Weights for layer 1
    weight1 = torch.tensor([[[1, 1, 1], [1, 0.1, 1], [1, 1, 1]],
                            [[1, 1, 1], [1, 1, 1], [1, 1, 1]]]).view(2, 1, 3, 3).float()
    b1 = torch.tensor([-3, -2]).float()
    # Weights for layer 2
    weight2 = torch.tensor([-10, 1]).view(1, 2, 1, 1).float()
    # Weights for layer 3
    s = 20
    weight3 = torch.tensor([2*s]).view(1, 1, 1, 1).float()
    b3 = torch.tensor([-s]).float()
  
    x = F.pad(x.float(), (1, 1, 1, 1), mode='circular')
    x = F.relu(F.conv2d(x, weight1, b1))
    x = F.relu(F.conv2d(x, weight2))
    x = torch.sigmoid(F.conv2d(x, weight3, b3))
    return x

In [ ]:
def get_data(train=True):
    if train:
        df = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/train.csv')
    else:
        df = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv')
    n = len(df)
    columns = df.columns
    delta = df.delta.to_numpy()
    stop_boards = df[[col for col in columns if col.startswith('stop')]].to_numpy()
    stop_boards = stop_boards.reshape(n, 1, 25, 25)
    if train:
        start_boards = df[[col for col in columns if col.startswith('start')]].to_numpy()
        start_boards = start_boards.reshape(n, 1, 25, 25)
        return delta, start_boards, stop_boards
    
    return delta, stop_boards

In [ ]:
train_deltas, train_start_boards, train_stop_boards = get_data()

In [ ]:
def advance(boards, deltas, num_iters=5):
    """ Advance boards that have delta less than num_iters.
    """
    advanced_boards = np.zeros_like(boards)
    for i, delta in enumerate(deltas):
        tmp_board = torch.tensor(boards[i,0,:,:]).reshape(1,1,25,25).int()
        for _ in range(num_iters - delta):
            tmp_board = evolve(tmp_board)
        advanced_boards[i,0,:,:] = tmp_board.int().numpy()
    return advanced_boards

In [ ]:
train_boards = advance(train_stop_boards, train_deltas)

In [ ]:
class ReverseNet(nn.Module):
    def __init__(self):
        super(ReverseNet, self).__init__()
        # in channels, out channels, kernel size
        self.conv1 = nn.Conv2d(1, 16, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ1 = nn.PReLU()
        self.conv2 = nn.Conv2d(16, 8, (5, 5), padding=(2, 2), padding_mode='circular')
        self.activ2 = nn.PReLU()
        self.conv3 = nn.Conv2d(8, 4, (3, 3), padding=(1, 1), padding_mode='circular')
        self.activ3 = nn.PReLU()
        self.conv4 = nn.Conv2d(4, 1, (3, 3), padding=(1, 1), padding_mode='circular')

    def forward(self, x):
        x = self.reverse(x)
        return x

    def reverse(self, x):
        x = self.activ1(self.conv1(x))
        x = self.activ2(self.conv2(x))
        x = self.activ3(self.conv3(x))
        x = torch.sigmoid(self.conv4(x))
        return x

In [ ]:
net = ReverseNet()
if torch.cuda.is_available():
    net.cuda()
print(net)

In [ ]:
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters()) #, lr=0.001, momentum=0.9)

In [ ]:
X = Variable(torch.tensor(train_boards).float(), requires_grad=True).cuda()
y = Variable(torch.tensor(train_start_boards).float()).cuda()
num_epochs = 60
batch_size = 500
for epoch in tqdm(range(num_epochs)): 
    permutation = torch.randperm(X.size()[0])
    running_loss = 0.0
    for i in range(0, X.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        batch = X[indices]
        target = y[indices]
        
        optimizer.zero_grad()
        outputs = net(batch)
        # loss = criterion(outputs, batch)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 0 and i > 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

In [ ]:
test_deltas, test_stop_boards = get_data(False)
test_boards = advance(test_stop_boards, test_deltas)

In [ ]:
X_test = Variable(torch.tensor(test_boards).float()).cuda()
with torch.no_grad():
    net.eval()
    predictions = (net(X_test) > 0.5).int()

In [ ]:
df = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/test.csv')
sample_submission = pd.read_csv('/kaggle/input/conways-reverse-game-of-life-2020/sample_submission.csv')

In [ ]:
pred_df = df[['id']].copy()
n = len(pred_df)
pred = predictions.cpu().numpy().reshape(n, 625)
for i in range(625):
    pred_df['start_{}'.format(i)] = pred[:,i]

In [ ]:
pred_df.to_csv('/kaggle/working/submission.csv', index=False)